# Kron's Method Tutorial using myPower
Kron's Method is one of the most used linearization of AC power flow. In this tutorial, you will be guided to make B matrix using Kron's Method and myPower original function, Kron's Slope Method. This work is under Apache License and act as pre-print of Kron's Slope Method publication (paper publication still on progress).

In this tutorial, we will use:

1. NRPF as Kron's Base
2. DCPF as evaluation target

It means that all losses are evaluated in evaluation target scenario (DCPF)

## import library

In [2]:
import os
import copy

import mypower as myp
from mypower.oc_api import oc_matpower

## set MATPOWER case_name

In [4]:
case_name = 'case9'

## initialize myPower

In [6]:
### start octave session for myPower
oc = oc_matpower()

In [8]:
### get index for unpacking myPowerCase
idx = myp.get_index()

In [10]:
### get slack generator index
mypc = oc.loadcase('case9')
mypc0 = myp.to_mypc0(mypc)
for bus_slack,val in zip(mypc0['bus'][:,idx['BUS_I']],mypc0['bus'][:,idx['BUS_TYPE']]):
    if val == idx['REF']:
        bus_slack = int(bus_slack)
        break
for num,val in enumerate(mypc0['gen'][:,idx['GEN_BUS']]):
    if int(val) == bus_slack:
        gen_slack = int(num)
        break
print(gen_slack)

0


## Newton Raphson Model (Also Kron's reference in making Kron's Matrix)

In [12]:
### run nrpf
mypc_nr = oc.runpf(case_name)


MATPOWER Version 7.1-dev, 08-Jul-2020 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.16 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transfo

In [14]:
### get losses for comparison
loss_nr = sum(mypc_nr['branch'][:, idx['PF']] + mypc_nr['branch'][:, idx['PT']])
print(loss_nr)

4.6410214744828835


In [16]:
### get slack generator power for comparison
gen_PG_slack_nr = mypc_nr['gen'][gen_slack,idx['PG']]
print(gen_PG_slack_nr)

71.64102147448243


## DCPF Model (Also Kron's input in using Kron's Coefficient)

In [18]:
### set mpoption to dcpf
mpopt = oc.mpoption(model='DC')
print(myp.pretty(mpopt))

{
    'v': 20.0,
    'model': 'DC',
    'pf': {
        'alg': 'NR',
        'current_balance': 0.0,
        'v_cartesian': 0.0,
        'tol': 1e-08,
        'nr': {
            'max_it': 10.0,
            'lin_solver': ''
        },
        'fd': {
            'max_it': 30.0
        },
        'gs': {
            'max_it': 1000.0
        },
        'radial': {
            'max_it': 20.0,
            'vcorr': 0.0
        },
        'enforce_q_lims': 0.0
    },
    'cpf': {
        'parameterization': 3.0,
        'stop_at': 'NOSE',
        'enforce_p_lims': 0.0,
        'enforce_q_lims': 0.0,
        'enforce_v_lims': 0.0,
        'enforce_flow_lims': 0.0,
        'step': 0.05,
        'step_min': 0.0001,
        'step_max': 0.2,
        'adapt_step': 0.0,
        'adapt_step_damping': 0.7,
        'adapt_step_tol': 0.001,
        'target_lam_tol': 1e-05,
        'nose_tol': 1e-05,
        'p_lims_tol': 0.01,
        'q_lims_tol': 0.01,
        'v_lims_tol': 0.0001,
        'flow_lims

In [20]:
### run dcpf 
mypc_dc = oc.runpf(case_name,mpopt)


MATPOWER Version 7.1-dev, 08-Jul-2020 -- DC Power Flow

Converged in 0.02 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0           0.0 to 0.0
Generators         3     On-line Capacity       820.0           0.0 to 0.0
Committed Gens     3     Generation (actual)    315.0               0.0
Loads              3     Load                   315.0               0.0
  Fixed            3       Fixed                315.0               0.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         0.00              0.00
Transformers       0     Branch Charging (inj)     -                0.0
Inter-ties         0     T

In [22]:
### get losses for comparison
loss_dc = sum(mypc_dc['branch'][:, idx['PF']] + mypc_dc['branch'][:, idx['PT']])
print(loss_dc)

0.0


In [24]:
### get slack generator power for comparison
gen_PG_slack_dc = mypc_dc['gen'][gen_slack,idx['PG']]
print(gen_PG_slack_dc)

66.99999999999996


## Kron's Method Model (Also Kron's Slope reference in making Kron's Slope Coefficient)

In [26]:
### make B using Kron's Method
'''Note: mypc_kr is not Kron's pf output, but pf used to make B_kron'''
B_kron,mypc_kr = myp.makeB_kron(case_name,oc=oc)
print(myp.pretty(B_kron))


MATPOWER Version 7.1-dev, 08-Jul-2020 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.12 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transfo

In [28]:
### get losses for comparison
#### total
B0_kron,B1_kron,B2_kron = B_kron['B0_kron'], B_kron['B1_kron'], B_kron['B2_kron']
loss_kr = myp.losses_kron(B0_kron,B1_kron,B2_kron,mypc_dc['gen'][:,idx['PG']],mypc_dc['baseMVA'])

#### individually
loss_kr0, loss_kr1, loss_kr2 = myp.losses_kron_detailed(B0_kron,B1_kron,B2_kron,mypc_dc['gen'][:,idx['PG']],mypc_dc['baseMVA'])

print('loss_kr:',loss_kr)
print('\tloss_kr0:',loss_kr0)
print('\tloss_kr1:',loss_kr1)
print('\tloss_kr2:',loss_kr2)

loss_kr: 4.663657720595333
	loss_kr0: 0.21534893428867388
	loss_kr1: -0.39227922587824804
	loss_kr2: 4.840588012184907


In [30]:
### get slack generator power for comparison
gen_PG_slack_kr = mypc_dc['gen'][gen_slack,idx['PG']] + loss_kr
print(gen_PG_slack_kr)

71.6636577205953


## Kron's Slope Model

In [32]:
### make B using Kron's Slope Method (Based on Taylor Series)
'''Note: mypc_krsl is not Kron's pf output, but pf used to make B_kron'''
B_kron_slope,mypc_krsl = myp.makeB_kron_slope(case_name,oc=oc)
print(myp.pretty(mypc_krsl['B_kron']))


MATPOWER Version 7.1-dev, 08-Jul-2020 -- AC Power Flow (Newton)

Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.16 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0        -900.0 to 900.0
Generators         3     On-line Capacity       820.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64             48.38
Transfo

In [34]:
### get losses for comparison
loss_krsl = myp.losses_kron_slope(B0_kron,B1_kron,B2_kron,B_kron_slope,mypc_krsl['B_kron']['PG_reff'],mypc_dc['gen'][:,idx['PG']],mypc_dc['baseMVA'])
print(loss_krsl)

4.643076985312411


In [36]:
### get slack generator power for comparison
gen_PG_slack_krsl = mypc_dc['gen'][gen_slack,idx['PG']] + loss_krsl
print(gen_PG_slack_krsl)

71.64307698531236


## Comparison

In [38]:
comparison = {
    'Losses': {
        'NRPF': loss_nr,
        'DCPF': loss_dc,
        'KRPF': loss_kr,
        'KRSLPF': loss_krsl
    },
    'Slack':{
        'NRPF': gen_PG_slack_nr,
        'DCPF': gen_PG_slack_dc,
        'KRPF': gen_PG_slack_kr,
        'KRSLPF': gen_PG_slack_krsl
    },
    'Losses Error':{
        'NRPF': (loss_nr - loss_nr) / loss_nr,
        'DCPF': (loss_dc - loss_nr) / loss_nr,
        'KRPF': (loss_kr - loss_nr) / loss_nr,
        'KRSLPF': (loss_krsl - loss_nr) / loss_nr
    },
    'Slack Error':{
        'NRPF': (gen_PG_slack_nr - gen_PG_slack_nr) / gen_PG_slack_nr,
        'DCPF': (gen_PG_slack_dc - gen_PG_slack_nr) / gen_PG_slack_nr,
        'KRPF': (gen_PG_slack_kr - gen_PG_slack_nr) / gen_PG_slack_nr,
        'KRSLPF': (gen_PG_slack_krsl - gen_PG_slack_nr) / gen_PG_slack_nr
    }
}
print(myp.pretty(comparison))

{
    'Losses': {
        'NRPF': 4.6410214744828835,
        'DCPF': 0.0,
        'KRPF': 4.663657720595333,
        'KRSLPF': 4.643076985312411
    },
    'Slack': {
        'NRPF': 71.64102147448243,
        'DCPF': 66.99999999999996,
        'KRPF': 71.6636577205953,
        'KRSLPF': 71.64307698531236
    },
    'Losses Error': {
        'NRPF': 0.0,
        'DCPF': -1.0,
        'KRPF': 0.004877427574275995,
        'KRSLPF': 0.00044290052110920606
    },
    'Slack Error': {
        'NRPF': 0.0,
        'DCPF': -0.0647816206268854,
        'KRPF': 0.00031596766275771243,
        'KRSLPF': 2.86918135396147e-05
    }
}
